In [ ]:
!pip install protobuf==3.20.3 mediapipe==0.10.9

In [ ]:
import os
import cv2
import numpy as np
from collections import defaultdict
import mediapipe as mp


# Define body part landmark groups (based on MediaPipe's 33 landmarks)
LANDMARK_GROUPS = {
    'arms': [11, 12, 13, 14, 15, 16],         # Shoulders to wrists
    'legs': [23, 24, 25, 26, 27, 28],         # Hips to ankles
    'core': [23, 24, 11, 12],                 # Hips and shoulders
    'upper_body': list(range(11)) + list(range(11, 17))  # Above hips
}

# Define exercise-specific landmark weighting (based on muscle involvement)
EXERCISE_WEIGHTS = {
    'bench press': {'arms': 0.5, 'core': 0.3, 'legs': 0.2},
    'barbell biceps curl': {'arms': 0.7, 'core': 0.2, 'legs': 0.1},
    'chest fly machine': {'arms': 0.5, 'core': 0.4, 'legs': 0.1},
    'deadlift': {'legs': 0.4, 'core': 0.4, 'arms': 0.2},
    'decline bench press': {'arms': 0.5, 'core': 0.3, 'legs': 0.2},
    'hammer curl': {'arms': 0.7, 'core': 0.2, 'legs': 0.1},
    'hip thrust': {'legs': 0.5, 'core': 0.4, 'arms': 0.1},
    'incline bench press': {'arms': 0.5, 'core': 0.3, 'legs': 0.2},
    'lat pulldown': {'arms': 0.5, 'core': 0.4, 'legs': 0.1},
    'lateral raise': {'arms': 0.6, 'core': 0.3, 'legs': 0.1},
    'leg extension': {'legs': 0.7, 'core': 0.2, 'arms': 0.1},
    'leg raises': {'core': 0.6, 'legs': 0.3, 'arms': 0.1},
    'plank': {'core': 0.7, 'arms': 0.2, 'legs': 0.1},
    'pull Up': {'arms': 0.5, 'core': 0.4, 'legs': 0.1},
    'push-up': {'arms': 0.5, 'core': 0.4, 'legs': 0.1},
    'romanian deadlift': {'legs': 0.5, 'core': 0.3, 'arms': 0.2},
    'russian twist': {'core': 0.6, 'arms': 0.2, 'legs': 0.2},
    'shoulder press': {'arms': 0.6, 'core': 0.3, 'legs': 0.1},
    'squat': {'legs': 0.6, 'core': 0.3, 'arms': 0.1},
    't bar row': {'arms': 0.4, 'core': 0.4, 'legs': 0.2},
    'tricep dips': {'arms': 0.6, 'core': 0.3, 'legs': 0.1},
    'tricep Pushdown': {'arms': 0.7, 'core': 0.2, 'legs': 0.1}
}

# Initialize MediaPipe Pose Model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,  # More accurate model
    min_detection_confidence=0.5
)

def calculate_weighted_confidence(landmarks, exercise_type):
    """Calculates visibility-weighted confidence per landmark"""
    weights = EXERCISE_WEIGHTS[exercise_type]
    weighted_scores = np.zeros(33)
    for group, weight in weights.items():
        for lm_idx in LANDMARK_GROUPS[group]:
            weighted_scores[lm_idx] = landmarks[lm_idx].visibility * weight
    return weighted_scores

def normalize_poses(poses):
    """Centers poses around the hip midpoint"""
    normalized_poses = []
    for pose in poses:
        landmarks = pose.reshape(-1, 4)
        left_hip = landmarks[23][:3]
        right_hip = landmarks[24][:3]
        hip_center = (left_hip + right_hip) / 2
        normalized_landmarks = landmarks.copy()
        normalized_landmarks[:, :3] -= hip_center
        normalized_poses.append(normalized_landmarks.flatten())
    return np.array(normalized_poses)

def calculate_velocity(poses):
    """Computes frame-to-frame velocity of poses"""
    velocity = np.zeros_like(poses)
    velocity[1:] = poses[1:] - poses[:-1]
    return velocity

def extract_poses_from_video(video_path, exercise_type, max_frames=45):
    """Processes a single video and extracts pose + velocity + confidence"""
    poses, confidence_scores, weighted_confidences = [], [], []
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Failed to open: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames-1, max_frames, dtype=int)

    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (640, 480))
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            confidence = np.mean([lm.visibility for lm in results.pose_landmarks.landmark])
            weighted_conf = calculate_weighted_confidence(results.pose_landmarks.landmark, exercise_type)
            if confidence > 0.4:
                lm_array = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
                poses.append(lm_array.flatten())
                confidence_scores.append(confidence)
                weighted_confidences.append(weighted_conf)
                continue

        # Fallback for low confidence or missing pose
        poses.append(np.zeros(33 * 4))
        confidence_scores.append(0.0)
        weighted_confidences.append(np.zeros(33))

    cap.release()

    # Pad if not enough frames
    while len(poses) < max_frames:
        poses.append(poses[-1])
        confidence_scores.append(0.0)
        weighted_confidences.append(weighted_confidences[-1])

    poses = np.array(poses)
    normalized = normalize_poses(poses)
    velocity = calculate_velocity(normalized)
    weighted_conf = np.array(weighted_confidences)
    final = np.concatenate([normalized, velocity, weighted_conf], axis=1)
    return final, np.array(confidence_scores)

def mirror_augment(poses):
    """Creates horizontally mirrored version of pose sequence"""
    mirrored = poses.copy()
    pose_part = mirrored[:, :33*4].reshape(-1, 33, 4)
    velocity = mirrored[:, 33*4:33*8].reshape(-1, 33, 4)
    weighted_conf = mirrored[:, 33*8:]

    # Mirror X coords
    pose_part[:, :, 0] = 1 - pose_part[:, :, 0]
    velocity[:, :, 0] *= -1

    # Swap left-right pairs
    pairs = [(1,2), (3,4), (5,6), (7,8), (9,10), (11,12), (13,14), (15,16),
             (23,24), (25,26), (27,28), (29,30), (31,32)]
    for i, j in pairs:
        pose_part[:, [i, j]] = pose_part[:, [j, i]]
        velocity[:, [i, j]] = velocity[:, [j, i]]
        weighted_conf[:, [i, j]] = weighted_conf[:, [j, i]]

    return np.concatenate([pose_part.reshape(poses.shape[0], -1),
                           velocity.reshape(poses.shape[0], -1),
                           weighted_conf], axis=1)

def prepare_and_save_dataset(base_folder, target_classes, save_dir, max_frames=45):
    """Processes the dataset, applies augmentation, saves as .npy"""
    X, y, confidences = [], [], []
    class_mapping = {}
    skipped_videos = defaultdict(int)

    print("Preparing dataset...")
    os.makedirs(save_dir, exist_ok=True)

    for root, _, files in os.walk(base_folder):
        class_name = os.path.basename(root)
        if class_name not in target_classes:
            continue
        if class_name not in class_mapping:
            class_mapping[class_name] = len(class_mapping)

        for file in files:
            if file.lower().endswith(('.mp4', '.avi', '.mov')):  # Check video formats
                try:
                    video_path = os.path.join(root, file)
                    poses, conf = extract_poses_from_video(video_path, class_name, max_frames)

                    if np.mean(conf) > 0.2:
                        X.append(poses)
                        y.append(class_mapping[class_name])
                        confidences.append(np.mean(conf))

                        X.append(mirror_augment(poses))
                        y.append(class_mapping[class_name])
                        confidences.append(np.mean(conf))
                    else:
                        skipped_videos[class_name] += 1
                except Exception as e:
                    print(f"Error processing {file}: {e}")
                    skipped_videos[class_name] += 1

    # Convert and save
    X, y, confidences = np.array(X), np.array(y), np.array(confidences)
    np.save(os.path.join(save_dir, 'X.npy'), X)
    np.save(os.path.join(save_dir, 'y.npy'), y)
    np.save(os.path.join(save_dir, 'confidences.npy'), confidences)
    np.save(os.path.join(save_dir, 'class_mapping.npy'), class_mapping)

    print(f"\nSaved to {save_dir}")
    print(f"X shape: {X.shape}")
    for name, idx in class_mapping.items():
        print(f"{name}: {(y == idx).sum()} samples")
    return X, y, class_mapping, confidences

if __name__ == "__main__":
    base_folder = "/content/drive/MyDrive/CVP/CV_PData"  # Folder with subfolders for each class
    save_dir = "/content/drive/MyDrive/CVP/processed_landmark_data"
    target_classes = list(EXERCISE_WEIGHTS.keys())  # all 22 exercise names

    X, y, class_mapping, confidences = prepare_and_save_dataset(base_folder, target_classes, save_dir)